# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


/Applications/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [7]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [8]:
# frequency table for prestige and whether or not someone was admitted
prestige_admit = pd.crosstab(index=df['prestige'],
                             columns=df['admit'])
prestige_admit.columns=["Admitted", "Not Admitted"]

prestige_admit

,Admitted,Not Admitted
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [9]:
df_dum = df.join(pd.get_dummies(df.prestige, prefix='prestige'))
df_dum.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: You need at least 2 or more to assign

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [10]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(df_dum.ix[:, 'prestige_1.0':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


/Applications/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [11]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
prestige1_admit = pd.crosstab(index=handCalc['prestige_1.0'],
                             columns="Count")
prestige1_admit.index=["Admitted", "Not Admitted"]

prestige1_admit

col_0,Count
Admitted,336
Not Admitted,61


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [12]:
def logit_func(odds):
    return np.log(odds)

def sigmoid_func(logit):
    return 1/(1+np.exp(-logit))

rank1_set = [
    336./397
]

for odds in rank1_set:
    print odds, sigmoid_func(logit_func(odds))

0.846347607053 0.458390177353


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [13]:
def logit_func(odds):
    return np.log(odds)

def sigmoid_func(logit):
    return 1/(1+np.exp(-logit))

rank2_set = [
    61./397
]

for odds in rank2_set:
    print odds, sigmoid_func(logit_func(odds))

0.153652392947 0.133187772926


#### 3.3 Calculate the odds ratio

In [20]:
prestige1_admit = pd.crosstab(index=handCalc['admit'],
                             columns="Count")
prestige1_admit.index=["Admitted", "Not Admitted"]

prestige1_admit

col_0,Count
Admitted,271
Not Admitted,126


In [22]:
OR_ADMIT = 126./271
OR_ADMIT

0.46494464944649444

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being admitted into this program is 0.465

#### 3.5 Print the cross tab for prestige_4

In [18]:
prestige4_admit = pd.crosstab(index=handCalc['prestige_4.0'],
                             columns=handCalc['admit'])
prestige4_admit.index=["Prestige4", "Not Prestige4"]

prestige4_admit

admit,0,1
Prestige4,216,114
Not Prestige4,55,12


#### 3.6 Calculate the OR 

In [26]:
prestige4_OR = 12./55
prestige4_OR

0.21818181818181817

#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted to the program coming from a school ranked 4 is 22%

## Part 4. Analysis

In [54]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(df_dum.ix[:, 'prestige_2.0':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [55]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [57]:
train_cols = data.columns[1:]
train_cols

Index([u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0',
       u'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [58]:
log = sm.Logit(data['admit'], data[train_cols])

log_fit = log.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [60]:
print log_fit.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 23 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        15:34:49   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [61]:
np.exp(log_fit.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [64]:
conf = log_fit.conf_int()
conf['OR'] = log_fit.params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: Prestige_2 OR of .50 means that the odds of being admitted to a program will decrease by 50% if the prestige of the school is 2 instead of 1.

#### 4.6 Interpret the OR of GPA

Answer: GPA OR of 2.18 means that a 1 unit increase in GPA score will increase the odds of being admitted by a factor of 2.18, and vice versa

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [65]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [85]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [86]:
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_rank = pd.get_dummies(combos['prestige'], prefix='prestige') # recreate the dummy variables
dummy_rank.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept'] # keep only what we need for making predictions
combos = combos[cols_to_keep].join(dummy_rank.ix[:, 'prestige_2.0':])

/Applications/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [87]:
combos.head()

,gre,gpa,prestige,intercept,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,1.0,1.0,0,0,0
1,220.0,2.260000,2.0,1.0,1,0,0
2,220.0,2.260000,3.0,1.0,0,1,0
3,220.0,2.260000,4.0,1.0,0,0,1
4,220.0,2.453333,1.0,1.0,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [89]:
combos['admit_pred'] = log_fit.predict(combos[train_cols])
print combos.head()

     gre       gpa  prestige  intercept  prestige_2.0  prestige_3.0  \
0  220.0  2.260000       1.0        1.0             0             0   
1  220.0  2.260000       2.0        1.0             1             0   
2  220.0  2.260000       3.0        1.0             0             1   
3  220.0  2.260000       4.0        1.0             0             0   
4  220.0  2.453333       1.0        1.0             0             0   

   prestige_4.0  admit_pred  
0             0    0.164173  
1             0    0.090492  
2             0    0.048977  
3             1    0.039890  
4             0    0.185907  


#### 5.3 Interpret findings for the last 4 observations

Answer: It looks as if a .19 increase in GPA, with all else constant (GRE and #1 rank), increases your chances of getting admitted by a value of .02.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.